In [1]:
import numpy as np
import pandas as pd
import os
import sys
import numpy as np
import yaml
import cv2
from scipy.interpolate import interp1d
from bisect import bisect
np.set_printoptions(precision=10)

In [2]:
scene = "office"
sequence= 7
sample = scene+"1-"+str(sequence)


inpath = os.path.join("/media/prakhar/BIG_BAG/Capstone/"+scene, sample)
print("Loading dataset from: "+inpath)
assert os.path.exists(inpath), "Path DNE: "+inpath

f1path = os.path.join(inpath,"fisheye1.txt")
f2path = os.path.join(inpath,"fisheye2.txt")

depthpath = os.path.join(inpath,"depth.txt")
gyropath = os.path.join(inpath,"t265_gyroscope.txt")
accpath = os.path.join(inpath,"t265_accelerometer.txt")

outpath = "/media/prakhar/BIG_BAG/Capstone/Loris_stereoi/"+sample
#associations file

sensorpath = "/media/prakhar/BIG_BAG/Capstone/"+scene+"/"+sample+"/sensors.yaml"
matpath = "/media/prakhar/BIG_BAG/Capstone/"+scene+"/"+sample+"/trans_matrix.yaml"

timestamp_dir = "/home/prakhar/CMU/Capstone/code/ORB_SLAM3/Examples/Stereo-Inertial/Loris_TimeStamps/"
timestamp_path = os.path.join(timestamp_dir, sample+".txt")

imu_dir = "/home/prakhar/CMU/Capstone/code/ORB_SLAM3/Examples/Stereo-Inertial/Loris_IMU/"
imu_path = os.path.join(imu_dir, sample+".txt")

if not os.path.exists(timestamp_dir):
    os.mkdir(timestamp_dir)
if not os.path.exists(imu_dir):
    os.mkdir(imu_dir)

Loading dataset from: /media/prakhar/BIG_BAG/Capstone/office/office1-7


In [3]:
sensor_data = cv2.FileStorage(sensorpath, cv2.FILE_STORAGE_READ)
mat_data = cv2.FileStorage(matpath, cv2.FILE_STORAGE_READ)

In [4]:
intr_acc = sensor_data.getNode("t265_accelerometer").getNode("imu_intrinsic").mat().reshape(3,4)
intr_gyro = sensor_data.getNode("t265_gyroscope").getNode("imu_intrinsic").mat().reshape(3,4)
print(intr_acc.shape, intr_gyro.shape)
print(intr_acc)
print(intr_gyro)

#all the noise values are variances while we need std
noise_acc = sensor_data.getNode("t265_accelerometer").getNode("noise_variances").mat().reshape(3,)
print("noise_acc_std",np.sqrt(noise_acc))
noise_gyro = sensor_data.getNode("t265_gyroscope").getNode("noise_variances").mat().reshape(3,)
print("noise_gyro_std",np.sqrt(noise_gyro))
bias_acc = sensor_data.getNode("t265_accelerometer").getNode("bias_variances").mat().reshape(3,)
print("bias_acc_std",np.sqrt(bias_acc))
bias_gyro = sensor_data.getNode("t265_gyroscope").getNode("bias_variances").mat().reshape(3,)
print("bias_gyro_std",np.sqrt(bias_gyro))


(3, 4) (3, 4)
[[ 1.0138926506  0.            0.           -0.051095333 ]
 [ 0.            1.0174480677  0.            0.2732152045]
 [ 0.            0.            1.0285693407  0.094960019 ]]
[[0.9956794977 0.           0.           0.0019203491]
 [0.           0.9980374575 0.           0.0021561817]
 [0.           0.           0.9926954508 0.0011997448]]
noise_acc_std [0.0081824478 0.0081824478 0.0081824478]
noise_gyro_std [0.0022689271 0.0022689271 0.0022689271]
bias_acc_std [0.0099999999 0.0099999999 0.0099999999]
bias_gyro_std [0.0007071068 0.0007071068 0.0007071068]


In [5]:
def parse_block(block):
    k1 = block[0].split(": ")[1]
    k2 = block[1].split(": ")[1]
    k3 = "".join(block[6:13]).replace(" ", "").split("data:")[1].replace("[","").replace("]","").split(",")
    k3 = np.array([float(l) for l in k3]).reshape((4,4))
    return k1,k2,k3

def get_trans_mats(matpath):
    with open(matpath,"r") as fp:
        data = [l[:-1] for l in fp.readlines()]
    # print(data)
    ls_blocks = []
    lsl = None
    for l in data:
        if '   -' == l:
            if lsl is not None:
                ls_blocks.append(lsl)
            lsl = []
        else:
            if lsl is not None:
                lsl.append(l)
    if lsl is not None:
        ls_blocks.append(lsl)
    ls_blocks = [parse_block(block) for block in ls_blocks]
    dict_trans = dict()
    for block in ls_blocks:
        k1,k2,k3 = block
        print(k1)
        dict_trans[k1] = dict()
    for block in ls_blocks:
        k1,k2,k3 = block
        dict_trans[k1][k2] = k3
    return dict_trans

In [6]:
dict_trans = get_trans_mats(matpath)
extr_acc = dict_trans['t265_fisheye1_optical_frame']['t265_accelerometer']
extr_gyro = dict_trans['t265_fisheye1_optical_frame']['t265_gyroscope']
extr_stereo = dict_trans['t265_fisheye1_optical_frame']['t265_fisheye2_optical_frame']

base_link
base_link
base_link
d400_color_optical_frame
d400_color_optical_frame
d400_color_optical_frame
t265_fisheye1_optical_frame
t265_fisheye1_optical_frame
t265_fisheye1_optical_frame


In [7]:
print(extr_acc)
print(extr_gyro)
print(extr_stereo)
assert np.allclose(extr_acc, extr_gyro)

[[-9.9992633142e-01  4.9341275781e-03  1.1089910930e-02  1.0699998587e-02]
 [-4.9039224757e-03 -9.9998419659e-01  2.7492007439e-03  7.2759576142e-12]
 [ 1.1103300579e-02  2.6946141507e-03  9.9993472576e-01 -2.9103830457e-11]
 [ 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  1.0000000000e+00]]
[[-9.9992633142e-01  4.9341275781e-03  1.1089910930e-02  1.0699998587e-02]
 [-4.9039224757e-03 -9.9998419659e-01  2.7492007439e-03  7.2759576142e-12]
 [ 1.1103300579e-02  2.6946141507e-03  9.9993472576e-01 -2.9103830457e-11]
 [ 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  1.0000000000e+00]]
[[ 9.9997103352e-01 -3.0859078733e-03  6.9576787768e-03  6.3976511359e-02]
 [ 3.1121161182e-03  9.9998809175e-01 -3.7591338255e-03  1.4826713595e-04]
 [-6.9459955824e-03  3.7806780409e-03  9.9996872932e-01 -3.9846837171e-04]
 [ 0.0000000000e+00  0.0000000000e+00  0.0000000000e+00  1.0000000000e+00]]


In [8]:
# extr_acc_inv = np.linalg.inv(extr_acc).flatten()
# print(list(extr_acc_inv))
# extr_stereo_inv = np.linalg.inv(extr_stereo).flatten()
# print(list(extr_stereo_inv))
print("accelerometrer extrisic matrix:\n",list(np.linalg.inv(extr_acc).flatten()))

print("right frame to left frame extrinsic matrix:\n",list(extr_stereo.flatten()))


accelerometrer extrisic matrix:
 [-0.9999263314167736, -0.004903922475695184, 0.011103300578872334, 0.010699210334022371, 0.004934127578069439, -0.999984196594937, 0.002694614150708582, -5.2795150761128155e-05, 0.011089910930018767, 0.0027492007438955656, 0.9999347257550536, -0.00011866200220366497, 0.0, 0.0, 0.0, 1.0]
right frame to left frame extrinsic matrix:
 [0.9999710335197896, -0.003085907873266471, 0.006957678776835974, 0.0639765113592, 0.003112116118197612, 0.9999880917521713, -0.0037591338254934705, 0.000148267135955, -0.006945995582404035, 0.0037806780408845384, 0.9999687293205325, -0.000398468371714, 0.0, 0.0, 0.0, 1.0]


In [9]:
def get_rgb_data(rgbpath):
    with open(rgbpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
        data = [[float(l[0]), l[0], l[1]] for l in data]
    return data

def get_aligned_depth_data(depthpath):
    with open(depthpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
        data = [[float(l[0]), l[0], 'aligned_'+l[1]] for l in data]
    return data

def get_gyroscope_data(gyropath, intr_gyro):
    with open(gyropath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
    head = data[0]
    data = data[1:]
    data = np.array([np.array([float(k) for k in l]) for l in data])
    datam = data[:,1:]
    data_scaled = np.transpose(intr_gyro[:3,:3]@datam.T) - intr_gyro[:,3]
    data[:,1:] = data_scaled
    return head, data
    
def get_accelerometer_data(accpath, intr_acc):
    with open(accpath, "r") as fp:
        data = fp.readlines()
        data = [l[:-1].split(" ") for l in data]
    head = data[0]
    data = data[1:]
    data = np.array([np.array([float(k) for k in l]) for l in data])
    datam = data[:,1:]
    data_scaled = np.transpose(intr_acc[:3,:3]@datam.T) - intr_acc[:,3]
    data[:,1:] = data_scaled
    return head, data


In [10]:
head, data_gyro = get_gyroscope_data(gyropath, intr_gyro)
#take every alternate row of gyro data
data_gyro = data_gyro[::2]
print(head, data_gyro.shape)
head, data_acc = get_accelerometer_data(accpath, intr_acc)
print(head, data_acc.shape)

['#Time', 'Gx', 'Gy', 'Gz'] (3789, 4)
['#Time', 'Ax', 'Ay', 'Az'] (2368, 4)


In [11]:
ind_max = bisect(data_gyro[:,0], data_acc[-1,0])
ind_min = bisect(data_gyro[:,0], data_acc[0,0])
data_gyro = data_gyro[ind_min:ind_max,:]

In [12]:
print("Gyro freq: ",100/(data_gyro[100,0] - data_gyro[0,0]))
print("accelereometer freq: ", 100/(data_acc[100,0] - data_acc[0,0]))


Gyro freq:  99.77610285756697
accelereometer freq:  62.35476696186154


In [13]:
#create interpolators
acc_interpolator = interp1d(data_acc[:,0], data_acc[:,1:], axis=0)
#interpolate at precise timesteps
acc_interp = acc_interpolator(data_gyro[:,0])

In [14]:
def write_IMU_readings(head, imu_mat, imu_path):
    with open(imu_path, "w") as fp:
        fp.write(head)
        for idr in range(imu_mat.shape[0]):
            reading = imu_mat[idr]
            ts = str(int(reading[0]))
            lsr = [str(x) for x in reading[1:]]
            lsr = [ts] + lsr
            line = ",".join(lsr) + "\n"
            fp.write(line)
    print("Done.")

#imu hed
head = "#timestamp [ns],w_RS_S_x [rad s^-1],w_RS_S_y [rad s^-1],w_RS_S_z [rad s^-1],a_RS_S_x [m s^-2],a_RS_S_y [m s^-2],a_RS_S_z [m s^-2]\n"
imu_mat = np.concatenate([data_gyro, acc_interp], axis=1)
imu_mat[:,0] *= 1e9

write_IMU_readings(head, imu_mat, imu_path)

imu_dir2 = os.path.join(outpath, "mav0/imu0/")

if not os.path.exists(imu_dir2):
    print("Creating dir: "+imu_dir2)
    os.makedirs(imu_dir2)

imu_path2 = os.path.join(imu_dir2, "data.csv")
write_IMU_readings(head, imu_mat, imu_path2)


Done.
Creating dir: /media/prakhar/BIG_BAG/Capstone/Loris_stereoi/office1-7/mav0/imu0/
Done.


In [15]:
print(imu_mat[:3])

[[ 1.5602449571e+18  5.5045114927e-03 -2.9769236003e-05  1.9728016232e-03
   1.2864987776e-01  9.7632174133e+00 -3.6710935683e-01]
 [ 1.5602449571e+18  2.0103955497e-04 -2.1561817266e-03  1.9728016232e-03
   1.2864987776e-01  9.7510566988e+00 -3.5481577649e-01]
 [ 1.5602449572e+18 -1.9203490810e-03 -8.5354194308e-03 -1.1997447582e-03
   1.3667183657e-01  9.7469467192e+00 -3.6693706107e-01]]


In [16]:
data_f1 = get_rgb_data(f1path)
data_f2 = get_rgb_data(f2path)

In [17]:
print("Dataset to be written at: "+outpath)

Dataset to be written at: /media/prakhar/BIG_BAG/Capstone/Loris_stereoi/office1-7


In [18]:
cam_path1 = os.path.join(outpath, "mav0/cam0/data/")
if not os.path.exists(cam_path1):
    print("Creating dir: "+cam_path1)
    os.makedirs(cam_path1)
cam_path2 = os.path.join(outpath, "mav0/cam1/data/")
if not os.path.exists(cam_path2):
    print("Creating dir: "+cam_path2)
    os.makedirs(cam_path2)

Creating dir: /media/prakhar/BIG_BAG/Capstone/Loris_stereoi/office1-7/mav0/cam0/data/
Creating dir: /media/prakhar/BIG_BAG/Capstone/Loris_stereoi/office1-7/mav0/cam1/data/


In [19]:
with open(timestamp_path, "w") as fp:
    for idx,(line1, line2) in enumerate(zip(data_f1, data_f2)):
        if line1[1] != line2[1]:
            print("Not same!!: ", line1[1], line2[1])
            continue
        
        tsstr = str(int(line1[0]*1e9))
        
        fp.write(tsstr+"\n")
        fname = tsstr+".png"
        dst1 = os.path.join(cam_path1, fname)
        dst2 = os.path.join(cam_path2, fname)
        
        src1 = os.path.join(inpath, line1[2])
        src2 = os.path.join(inpath, line2[2])
        command1 = "ln -sf "+src1+" "+dst1
        command2 = "ln -sf "+src2+" "+dst2
#         print(command1)
#         print(command2)
#         break
        os.system(command1)
        os.system(command2)
        if (idx+1)%100 == 0:
            print(idx+1,len(data_f1))

100 1140
200 1140
300 1140
400 1140
500 1140
600 1140
700 1140
800 1140
900 1140
1000 1140
1100 1140


In [ ]:
"/media/prakhar/BIG_BAG/Capstone/office/office1-1/fisheye1/1560000083.949067.png"
"/media/prakhar/BIG_BAG/Capstone/office/office1-1/fisheye2/1560000083.949067.png"